In [26]:
import pandas as pd
import numpy as np
import datetime


In [27]:
t0 = datetime.datetime.now()

data = pd.read_excel ("Test_Pandas.xlsx")
t1 = datetime.datetime.now()
print('Execution time: {}'.format(t1 - t0))
data.head()

Execution time: 0:01:15.621117


,itemid,shopid,item_name,item_description,item_variation,price,stock,category,cb_option,is_preferred,sold_count,item_creation_date
0,1925574,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355333 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:24:42
1,1925617,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355888 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:26:37
2,943600,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-25 21:02:13
3,1064405,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-30 20:16:07
4,20046620,760000,101% AUTHENTIC BASEBALL CAPS,"PREORDER Takes about 23 weeks to arrive, will ...","{NEWERA BLACK: 35.0, NIKE SWOOSH DENIM: 35.0, ...",35.0,300,Women's Apparel,0,0,0,2016-05-09 01:14:34


In [23]:
nb_shop = data['shopid'].nunique()
print(nb_shop)

7856


In [24]:
shop_duplicate = data[data.duplicated(['shopid','Itemid','item_description','price'])]

KeyError: Index(['Itemid'], dtype='object')

In [15]:
data = pd.read_csv("Test_Pandas_1.csv",)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 39: invalid continuation byte

In [ ]:
columns = data.columns
# Number unique shops:
nb_shop = len(data['Shopid'].nuqniue)

# product have sold count
nb_product =data[data['sold_count']=0]